[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/monitoring/llms/openai-assistant/openai_assistant.ipynb)


# <a id="top">OpenAI assistant monitoring</a>

This notebook illustrates how to get started monitoring [OpenAI Assistants](https://platform.openai.com/docs/assistants/overview) with Openlayer.

In [ ]:
!pip install openlayer

## 1. Set the environment variables

In [ ]:
import openai
import os

# Set the environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"
os.environ["OPENLAYER_API_KEY"] = "YOUR_OPENLAYER_API_KEY_HERE"
os.environ["OPENLAYER_PROJECT_NAME"] = "YOUR_OPENLAYER_PROJECT_NAME_HERE"

## 2. Instantiate the monitor

In [ ]:
from openlayer import llm_monitors

openai_client = openai.OpenAI()
monitor = llm_monitors.OpenAIMonitor(client=openai_client, publish=True)

## 3. Create assistant, thread, and run it

In [ ]:
# Create the assistant
assistant = openai_client.beta.assistants.create(
    name="Data visualizer",
    description="You are great at creating and explaining beautiful data visualizations.",
    model="gpt-4",
    tools=[{"type": "code_interpreter"}],
)

In [ ]:
# Create a thread
thread = openai_client.beta.threads.create(
    messages=[
        {
        "role": "user",
        "content": "Create a data visualization of the american GDP.",
        }
    ]
)

In [ ]:
# Run assistant on thread
run = openai_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [ ]:
import time

# Keep polling the run results
while run.status != "completed":
    run = openai_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Monitor the run with the Openlayer `monitor`. If complete, the thread is sent to Openlayer
    monitor.monitor_thread_run(run)

    time.sleep(5)